In [1]:
import pandas as pd

In [2]:

df = pd.read_csv(
    "groupby-data/airqual.csv",
    parse_dates=[["Date", "Time"]],
    na_values=[-200],
    usecols=["Date", "Time", "CO(GT)", "T", "RH", "AH"]
).rename(
    columns={
        "CO(GT)": "co",
        "Date_Time": "tstamp",
        "T": "temp_c",
        "RH": "rel_hum",
        "AH": "abs_hum",
    }
).set_index("tstamp")

/var/folders/8g/vflxk7vd1sj2bbxxydl87kbw0000gn/T/ipykernel_6238/2257211669.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(


In [4]:
df.head()

,co,temp_c,rel_hum,abs_hum
tstamp,,,,
2004-03-10 18:00:00,2.6,13.6,48.9,0.7578
2004-03-10 19:00:00,2.0,13.3,47.7,0.7255
2004-03-10 20:00:00,2.2,11.9,54.0,0.7502
2004-03-10 21:00:00,2.2,11.0,60.0,0.7867
2004-03-10 22:00:00,1.6,11.2,59.6,0.7888


In [6]:
day_names = df.index.day_name()
df.groupby(day_names)["co"].mean()

tstamp
Friday       2.543041
Monday       2.016741
Saturday     1.861077
Sunday       1.438069
Thursday     2.455505
Tuesday      2.382267
Wednesday    2.400787
Name: co, dtype: float64

In [7]:
hr = df.index.hour
df.groupby([day_names, hr])["co"].mean().rename_axis(["dow", "hr"])

dow        hr
Friday     0     1.936170
           1     1.608511
           2     1.172340
           3     0.887234
           4     0.823333
                   ...   
Wednesday  19    4.146809
           20    3.844681
           21    2.897872
           22    2.102128
           23    1.938298
Name: co, Length: 168, dtype: float64

In [9]:
bins = pd.cut(df["temp_c"], bins=3, labels=("cool", "warm", "hot"))
df[["rel_hum", "abs_hum"]].groupby(by=bins, observed=False).agg(["mean", "median"])

rel_hum          abs_hum        
             mean median      mean  median
temp_c                                    
cool    57.651452   59.2  0.665874  0.6581
warm    49.382716   49.3  1.182894  1.1452
hot     24.994334   24.1  1.292958  1.2742

In [10]:
bins.head()

tstamp
2004-03-10 18:00:00    cool
2004-03-10 19:00:00    cool
2004-03-10 20:00:00    cool
2004-03-10 21:00:00    cool
2004-03-10 22:00:00    cool
Name: temp_c, dtype: category
Categories (3, object): ['cool' < 'warm' < 'hot']

In [11]:
df.groupby([df.index.year, df.index.quarter])["co"].agg(
    ["max", "min"]
).rename_axis(
    ["year", "quarter"]
)

max  min
year quarter           
2004 1         8.1  0.3
     2         7.3  0.1
     3         7.5  0.1
     4        11.9  0.1
2005 1         8.7  0.1
     2         5.0  0.3

In [15]:
# "Q" stands for "quarterly"
df.resample(rule="Q")["co"].agg(["max", "min"])

,max,min
tstamp,,
2004-03-31,8.1,0.3
2004-06-30,7.3,0.1
2004-09-30,7.5,0.1
2004-12-31,11.9,0.1
2005-03-31,8.7,0.1
2005-06-30,5.0,0.3
